In [79]:
#imports
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import pandas as pd

In [80]:
#set up api key variables
keysfile = open('apikeys','r')
cl_id = keysfile.readline().strip()
cl_sec = keysfile.readline().strip()

#set up spotify object
auth_manager = SpotifyClientCredentials(client_id=cl_id, client_secret=cl_sec)
sp = spotipy.Spotify(auth_manager=auth_manager)


In [81]:
#requests to get all of moby's albums
moby_uri = 'spotify:artist:3OsRAKCvk37zwYcnzRf5XF'

results = sp.artist_albums(moby_uri, album_type='album')
albums = results['items']
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])


In [82]:
# only released in the US (which removes a bunch of duplicates)
albums = [album for album in albums if 'US' in album['available_markets'] ]

# remove duplicates
temp=[]
for album in albums:
    if album['name'].lower() not in [a['name'].lower() for a in temp]:
        temp.append(album)
albums = temp

# remove specific albums
omit = ["Reprise (Commentary Version)","Porcelain (Gekürzte Lesung)","Destroyed","Wait for Me","Wait for Me (Deluxe Version)",
    'Everything Was Beautiful, And Nothing Hurt (The EastWest Sessions)','Innocents: Live at the Fonda, LA','Wait for Me (Ambient)',
    'Wait for Me. Remixes!','Wait For Me (Bonus Track Version)','All Visible Objects (Quarantine DJ Set)']
albums = [album for album in albums if album['name'] not in omit]

# for album in albums:
#     print(album['name'])
# len(albums)


In [83]:
#categories for all of the albums
album_cat = {
    '18 & 18 B-Sides':'Studio',
    'All Visible Objects':'Studio',
    'All Visible Objects [Remixed : DJ Set]':'Remix',
    'Ambient':'Ambient',
    'Animal Rights':'Studio',
    'Destroyed Remixed':'Remix',
    'Destroyed [Deluxe Edition]':'Studio',
    'Early Underground':'Studio',
    'Everything Is Wrong':'Studio',
    'Everything Was Beautiful, and Nothing Hurt':'Studio',
    'Everything Was Beautiful, and Nothing Hurt (Remixes)':'Remix',
    'Hotel':'Studio',
    'Hotel Ambient':'Ambient',
    'I Like to Score':'Studio',
    'Innocents':'Studio',
    'Last Night':'Studio',
    'Last Night Remixed':'Remix',
    'Live Ambient Improvised Recordings, Vol. 1':'Ambient',
    'Long Ambients 1: Calm. Sleep':'Ambient',
    'Long Ambients Two':'Ambient',
    'Moby':'Studio',
    'More Fast Songs About The Apocalypse':'Studio',
    'Next Is the E':'Studio',
    'Play & Play: B Sides':'Studio',
    'Remixes':'Remix',
    'Reprise':'Studio',
    'Reprise - Remixes':'Remix',
    'Songs 1993 - 1998':'Studio',
    'The Remixes':'Remix',
    'These Systems Are Failing':'Studio',
    'Wait For Me Ambient':'Ambient',
    'Wait For Me Remixes!':'Remix',
    'Wait for Me (Deluxe Edition)':'Studio'
    }

In [84]:
#get all the track ids and names from all of the albums
track_ids = []
track_names = []
track_albums = []
track_categories = []

for album in albums:
    tracks = sp.album_tracks(album['id'])['items']
    track_ids.extend([track['id'] for track in tracks])
    track_names.extend([track['name'] for track in tracks])
    track_albums.extend([album['name']] * len(tracks))
    track_categories.extend([album_cat[album['name']]] * len(tracks))


In [71]:
data = []
indexes = list(range(0,len(track_ids),100)) + [len(track_ids)]
for i in range(len(indexes)-1):
    data.extend(sp.audio_features(track_ids[indexes[i]:indexes[i+1]]))


In [89]:
#put it in a dataframe with names, drop unnecessary fields, export
df = pd.DataFrame(data)
df.insert(0,'name',track_names,True)
df.insert(1,'Album',track_albums,True)
df.insert(2,'Category',track_categories,True)
df.drop(['type','uri','track_href','analysis_url'], axis=1, inplace=True)
df.rename({'danceability':'Danceability'},axis='columns',inplace=True)

In [90]:

df.to_csv('msd2.csv')